In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def single_lamp_light_distribution(road_length,number_of_points):  #  this function gets iliuminaion distribution of single light

    x = np.linspace(-road_length/2,road_length/2,number_of_points).astype(int)    
    distribution = 3**(-(x / 90)**2) 

    for i in range(len(distribution)):
        if distribution[i] < 0.01:
            distribution[i] = 0
    distribution = np.trim_zeros(distribution)
    plotis = int( (len(distribution) +1)/2)

    return distribution, plotis   

def all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length): #  when given indexes with light that are not working we can calculate total illiumination of the street

    intensity = np.zeros(number_of_points)

    for i in range(int(road_length/20+1)):
        if i in non_working_street_lights:  #  skips if there are no light from street light 
            continue
        lamp_influence_start = i*20 - plotis 
        if lamp_influence_start < 0:
            corection = lamp_influence_start * (-1)  #  "corection" is needed that function wont calculate intensity before street starts. 
        else:
            corection = 0

        if i*20 + plotis  < number_of_points:   
            for y in distribution[corection:]:
                intensity[lamp_influence_start + corection] +=  y #  adds every light illiumination to total illiumination
                lamp_influence_start += 1 
        else:
            back_corection = number_of_points - (i*20 + plotis-1 )
            for y in distribution[corection:back_corection]:
                intensity[lamp_influence_start + corection] +=  y
                lamp_influence_start += 1 


    

    return intensity

def find_min_index(intensity, non_working_street_lights): #  simple findind first minimal intensity. But takes point only where lights are (every 20m)
    min = 8
    for i in non_working_street_lights:
        if intensity[i*20] < min:
            min = intensity[i*20]
            min_index = i
    return min_index

def plot_light_intensity(intensity,road_length,number_of_points):

    X = np.linspace(0,road_length,number_of_points).astype(int)
    plt.plot(X,intensity)
    plt.xlim(0, number_of_points)
    plt.xlabel("Street lenght, m")
    plt.ylabel("Street illiumination, AU")
    plt.ylim(0, 8)

def find_index_of_darkest_street_light(road_length, number_of_points,  non_working_street_lights):

    distribution, plotis = single_lamp_light_distribution(road_length,number_of_points)

    intensity = all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length)

    min_index = find_min_index(intensity, non_working_street_lights)
    
    #plot_light_intensity(intensity)


    return min_index

def minimum_lapms_needed(non_working_street_lights, number_of_points, road_length):
    
    distribution, plotis = single_lamp_light_distribution(road_length,number_of_points)

    intensity = all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length)

    to_light_up = []  #  for lights that we need to change
    non_working_dark = [] #  for calculations



    # what this function does: 
    # It takes concecutive lights that are broken and puts it in "non_working_dark". Then lights up light in the middle and checks if illiumination in that section now more then 1. 
    # If YES then it moves to another section.
    # If when light is lit up there are still spots with intensity smaler then 1, then it lights up 2 light, and if doens help then 3 lights and so on, until illiumination in that section is more then 1. 
    for i in non_working_street_lights:
        
        number_to_lightup = 1
        if intensity[i*20] < 1:
            non_working_dark.append(i)


        elif len(non_working_dark) > 0:
            
            light_it_up_index = int(len(non_working_dark)/(number_to_lightup+1)) 
            numbers = non_working_dark[light_it_up_index]
            

            while intensity[0:i*20].min() < 1:

                
                light_it_up_index = int(len(non_working_dark)/(number_to_lightup+1))
                
                numbers = non_working_dark[light_it_up_index]

                for y in range(1, number_to_lightup+1):
                    
                    numbers = non_working_dark[light_it_up_index * y]
                    non_working_street_lights.remove(numbers)
                    to_light_up.append(numbers)
                    intensity = all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length)

                if intensity[0:i*20].min() < 1:
                    for y in range(1, number_to_lightup+1):
                        numbers = non_working_dark[light_it_up_index * y]
                        non_working_street_lights.append(numbers)
                        to_light_up.remove(numbers)
                        non_working_street_lights.sort()
                        

                number_to_lightup += 1
            
            non_working_dark = []
            intensity = all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length)
            #plot_light_intensity(intensity)

    to_light_up.sort()

    return  to_light_up, intensity

In [ ]:
road_length = 2000
number_of_points = road_length + 1
non_working_street_lights = [ 1, 2, 3, 5, 9, 10, 11, 12,  13, 16, 17, 18, 20, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 42, 43, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]
distribution, plotis = single_lamp_light_distribution(road_length,number_of_points)

old_intensity = all_lamp_light_distribution(distribution, plotis, number_of_points, non_working_street_lights, road_length)

In [ ]:
min_index = find_index_of_darkest_street_light(road_length, number_of_points,  non_working_street_lights)

print(f"Minimal light iliumination is at {min_index} street lamp, and need to be replaced")

In [ ]:
nr_to_light_up, new_intensity =  minimum_lapms_needed(non_working_street_lights, number_of_points, road_length)

print(f"Minimum amount of street lapms to be replaced: {len(nr_to_light_up)} \nIndexes of those lamps are: {nr_to_light_up}\n\n")

print("Street illiumination distribution before changing lamps:")

plot_light_intensity(old_intensity, road_length, number_of_points)
plt.show()
print("Street illiumination distribution after changing lamps:")
plot_light_intensity(new_intensity, road_length, number_of_points)